# Testing GaugeModel

## Imports

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import pickle

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from collections import namedtuple

#from models.model import GaugeModel
#from models.gauge_model import GaugeModel
from loggers.train_logger import TrainLogger
from loggers.run_logger import RunLogger
from trainers.trainer import GaugeModelTrainer
from plotters.gauge_model_plotter import GaugeModelPlotter
from plotters.leapfrog_plotters import LeapfrogPlotter
from runners.runner import GaugeModelRunner
from main import create_config, train_setup
from config import PARAMS
import utils.file_io as io

import matplotlib as mpl
import matplotlib.pyplot as plt

%autoreload 2
%matplotlib notebook
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

In [2]:
from models.gauge_model import GaugeModel
from models.params import GAUGE_PARAMS
from loggers.train_logger import TrainLogger
from main import create_config
from config import NP_FLOAT

print(f'GAUGE_PARAMS:\n')
_ = [print(f'  {k}: {v}') for k, v in GAUGE_PARAMS.items()]

params, hooks = train_setup(GAUGE_PARAMS, log_file=None)
model = GaugeModel(params)

train_logger = TrainLogger(model, params['log_dir'],
                           logging_steps=10, summaries=params['summaries'])

config, params = create_config(params)
net_weights_init = [1., 1., 1.]
samples_init = np.reshape(np.array(model.lattice.samples, dtype=NP_FLOAT),
                          (model.num_samples, model.x_dim))
beta_init = model.beta_init

checkpoint_dir = os.path.join(model.log_dir, 'checkpoints')
io.check_else_make_dir(checkpoint_dir)

sess_kwargs = {
    'checkpoint_dir': checkpoint_dir,
    'hooks': hooks,
    'config': config,
    'save_summaries_secs': None,
    'save_summaries_steps': None,
}

global_var_init = tf.global_variables_initializer()
local_var_init = tf.local_variables_initializer()
uninited = tf.report_uninitialized_variables()

sess = tf.train.MonitoredTrainingSession(**sess_kwargs)
tf.keras.backend.set_session(sess)

trainer = GaugeModelTrainer(sess, model, logger=train_logger)

GAUGE_PARAMS:

  space_size: 8
  time_size: 8
  link_type: U1
  dim: 2
  num_samples: 128
  rand: True
  num_steps: 5
  eps: 0.2
  fixed_beta: False
  hmc_beta: None
  hmc_eps: None
  beta_init: 2.0
  beta_final: 5.0
  inference: False
  beta_inference: None
  lr_init: 0.001
  lr_decay_steps: 1000
  lr_decay_rate: 0.96
  warmup_lr: False
  train_steps: 5000
  run_steps: 10000
  network_arch: generic
  num_hidden1: 50
  num_hidden2: 50
  use_bn: False
  dropout_prob: 0.5
  clip_value: 0.0
  summaries: True
  save_samples: False
  save_lf: True
  eps_fixed: False
  hmc: False
  loss_scale: 0.1
  std_weight: 1.0
  aux_weight: 1.0
  charge_weight: 0.0
  metric: cos_diff
  trace: False
  profiler: False
  gpu: False
  horovod: False
  comet: False
  restore: False
  theta: False
  num_intra_threads: 0
  float64: False
  using_hvd: False
--------------------------------------------------------------------------------
Starting training using L2HMC algorithm...
Creating directory: /Users/safor

In [3]:
train_kwargs = {
    'samples_np': samples_init,
    'beta_np': beta_init,
    'net_weights': net_weights_init,
    'print_steps': 1.,
}

trainer.train(5000, **train_kwargs)

-----------------------------------------------------------------------------------------------------------------
    STEP       LOSS     t/STEP    % ACC      EPS       BETA     ACTION     PLAQ    (EXACT)      dQ        LR    
-----------------------------------------------------------------------------------------------------------------
    0/5000     -1054     13.26    0.6431     0.201       2       63.57   0.006792   0.6978    0.4141     0.001   
    0/5000     -1054     13.26    0.6431     0.201       2       63.57   0.006792   0.6978    0.4141     0.001   
    1/5000    -983.8    0.3945    0.5453     0.202       2       47.89    0.2517    0.6978    0.2266     0.001   
    1/5000    -983.8    0.3945    0.5453     0.202       2       47.89    0.2517    0.6978    0.2266     0.001   
    2/5000     -1011    0.2527     0.62      0.203       2       39.62     0.381    0.6979    0.2031   0.0009999 
    2/5000     -1011    0.2527     0.62      0.203       2       39.62     0.381    0.697

   35/5000     -1184    0.5968     0.727    0.2353     2.008     18.73    0.7073    0.6992    0.1484   0.0009986 
   35/5000     -1184    0.5968     0.727    0.2353     2.008     18.73    0.7073    0.6992    0.1484   0.0009986 
   36/5000     -1202    0.2676    0.7204    0.2363     2.009     18.74    0.7072    0.6992     0.125   0.0009985 
   36/5000     -1202    0.2676    0.7204    0.2363     2.009     18.74    0.7072    0.6992     0.125   0.0009985 
   37/5000     -1155    0.3122    0.6443    0.2372     2.009     18.73    0.7073    0.6992    0.07031  0.0009985 
   37/5000     -1155    0.3122    0.6443    0.2372     2.009     18.73    0.7073    0.6992    0.07031  0.0009985 
   38/5000     -1194    0.3708    0.6974    0.2381     2.009     18.61    0.7092    0.6993    0.09375  0.0009984 
   38/5000     -1194    0.3708    0.6974    0.2381     2.009     18.61    0.7092    0.6993    0.09375  0.0009984 
   39/5000     -1207    0.5805    0.6731     0.239     2.009     18.8     0.7063    0.69

   71/5000     -1347    0.4293    0.7048    0.2645     2.017     19.23    0.6996    0.7006    0.1406   0.0009971 
   71/5000     -1347    0.4293    0.7048    0.2645     2.017     19.23    0.6996    0.7006    0.1406   0.0009971 
   72/5000     -1327    0.3504    0.6752    0.2651     2.017     19.51    0.6951    0.7006    0.1641   0.0009971 
   72/5000     -1327    0.3504    0.6752    0.2651     2.017     19.51    0.6951    0.7006    0.1641   0.0009971 
   73/5000     -1322    0.1694    0.6436    0.2658     2.018     19.39    0.6971    0.7007    0.08594  0.000997  
   73/5000     -1322    0.1694    0.6436    0.2658     2.018     19.39    0.6971    0.7007    0.08594  0.000997  
   74/5000     -1349    0.5036    0.6909    0.2665     2.018     19.47    0.6958    0.7007    0.1016   0.000997  
   74/5000     -1349    0.5036    0.6909    0.2665     2.018     19.47    0.6958    0.7007    0.1016   0.000997  
   75/5000     -1353     0.249    0.7049    0.2672     2.018     19.29    0.6986    0.70

  104/5000     -1409    0.3891    0.6534     0.284     2.025     18.98    0.7034    0.7019    0.1172   0.0009958 
  104/5000     -1409    0.3891    0.6534     0.284     2.025     18.98    0.7034    0.7019    0.1172   0.0009958 
  105/5000     -1404    0.3192    0.6145    0.2845     2.026     19.2     0.6999    0.7019    0.1016   0.0009957 
  105/5000     -1404    0.3192    0.6145    0.2845     2.026     19.2     0.6999    0.7019    0.1016   0.0009957 
  106/5000     -1409    0.2813    0.6285    0.2848     2.026     19.28    0.6988     0.702    0.09375  0.0009957 
  106/5000     -1409    0.2813    0.6285    0.2848     2.026     19.28    0.6988     0.702    0.09375  0.0009957 
  107/5000     -1404    0.3844    0.6191    0.2852     2.026     19.36    0.6974     0.702     0.125   0.0009956 
  107/5000     -1404    0.3844    0.6191    0.2852     2.026     19.36    0.6974     0.702     0.125   0.0009956 
  108/5000     -1478    0.3658    0.6444    0.2856     2.026     19.63    0.6933     0.7

  140/5000     -1481    0.4101    0.6265    0.2954     2.034     19.71     0.692    0.7033     0.125   0.0009943 
  140/5000     -1481    0.4101    0.6265    0.2954     2.034     19.71     0.692    0.7033     0.125   0.0009943 
  141/5000     -1488    0.3274    0.6595    0.2956     2.034     19.8     0.6907    0.7033    0.1328   0.0009943 
  141/5000     -1488    0.3274    0.6595    0.2956     2.034     19.8     0.6907    0.7033    0.1328   0.0009943 
  142/5000     -1455    0.3998    0.5964    0.2957     2.035     19.79    0.6908    0.7034    0.1328   0.0009942 
  142/5000     -1455    0.3998    0.5964    0.2957     2.035     19.79    0.6908    0.7034    0.1328   0.0009942 
  143/5000     -1464    0.2793    0.5922    0.2959     2.035     20.04    0.6868    0.7034    0.1719   0.0009942 
  143/5000     -1464    0.2793    0.5922    0.2959     2.035     20.04    0.6868    0.7034    0.1719   0.0009942 
  144/5000     -1411    0.2212    0.6209    0.2961     2.035     20.05    0.6867    0.70

...

  841/5000     -1606    0.2704    0.7293    0.3013     2.224     17.63    0.7245    0.7314    0.1562   0.0009663 
  841/5000     -1606    0.2704    0.7293    0.3013     2.224     17.63    0.7245    0.7314    0.1562   0.0009663 
  842/5000     -1613    0.2209    0.7777    0.3014     2.225     17.41     0.728    0.7314    0.1016   0.0009662 
  842/5000     -1613    0.2209    0.7777    0.3014     2.225     17.41     0.728    0.7314    0.1016   0.0009662 
  843/5000     -1584    0.6506    0.7004    0.3015     2.225     17.33    0.7292    0.7315    0.07031  0.0009662 
  843/5000     -1584    0.6506    0.7004    0.3015     2.225     17.33    0.7292    0.7315    0.07031  0.0009662 
  844/5000     -1576    0.6159    0.7429    0.3016     2.225     17.48    0.7269    0.7315    0.0625   0.0009661 
  844/5000     -1576    0.6159    0.7429    0.3016     2.225     17.48    0.7269    0.7315    0.0625   0.0009661 
  845/5000     -1585    0.5425     0.751    0.3017     2.226     17.79     0.722    0.73

  877/5000     -1597    0.4363    0.7076    0.3038     2.235     17.23    0.7307    0.7329    0.1094   0.0009648 
  877/5000     -1597    0.4363    0.7076    0.3038     2.235     17.23    0.7307    0.7329    0.1094   0.0009648 
  878/5000     -1531    0.2935    0.6888    0.3039     2.236     17.3     0.7297    0.7329    0.1094   0.0009648 
  878/5000     -1531    0.2935    0.6888    0.3039     2.236     17.3     0.7297    0.7329    0.1094   0.0009648 
  879/5000     -1595    0.6241    0.7087    0.3039     2.236     17.12    0.7325    0.7329    0.07812  0.0009648 
  879/5000     -1595    0.6241    0.7087    0.3039     2.236     17.12    0.7325    0.7329    0.07812  0.0009648 
  880/5000     -1565    0.3437    0.7649     0.304     2.236     17.36    0.7288     0.733    0.09375  0.0009647 
  880/5000     -1565    0.3437    0.7649     0.304     2.236     17.36    0.7288     0.733    0.09375  0.0009647 
  881/5000     -1612    0.3851    0.7575    0.3039     2.236     17.25    0.7304     0.7

  910/5000     -1618     0.324    0.7842     0.304     2.245     17.47     0.727    0.7342    0.1484   0.0009635 
  910/5000     -1618     0.324    0.7842     0.304     2.245     17.47     0.727    0.7342    0.1484   0.0009635 
  911/5000     -1544    0.3987    0.7104     0.304     2.245     17.54     0.726    0.7342    0.09375  0.0009635 
  911/5000     -1544    0.3987    0.7104     0.304     2.245     17.54     0.726    0.7342    0.09375  0.0009635 
  912/5000     -1602    0.5954    0.7111     0.304     2.246     17.66    0.7241    0.7343    0.1328   0.0009635 
  912/5000     -1602    0.5954    0.7111     0.304     2.246     17.66    0.7241    0.7343    0.1328   0.0009635 
  913/5000     -1626    0.4787    0.7649    0.3038     2.246     17.63    0.7245    0.7343    0.1172   0.0009634 
  913/5000     -1626    0.4787    0.7649    0.3038     2.246     17.63    0.7245    0.7343    0.1172   0.0009634 
  914/5000     -1552    0.5298    0.7025    0.3037     2.246     17.14    0.7322    0.73

  947/5000     -1567    0.2661     0.67     0.3042     2.256     17.25    0.7304    0.7357    0.09375  0.0009621 
  947/5000     -1567    0.2661     0.67     0.3042     2.256     17.25    0.7304    0.7357    0.09375  0.0009621 
  948/5000     -1618    0.4197    0.7523    0.3043     2.257     17.52    0.7263    0.7357    0.09375  0.000962  
  948/5000     -1618    0.4197    0.7523    0.3043     2.257     17.52    0.7263    0.7357    0.09375  0.000962  
  949/5000     -1579    0.4763     0.705    0.3044     2.257     16.98    0.7347    0.7358    0.09375  0.000962  
  949/5000     -1579    0.4763     0.705    0.3044     2.257     16.98    0.7347    0.7358    0.09375  0.000962  
  950/5000     -1581    0.2028    0.7206    0.3043     2.257      17      0.7343    0.7358    0.1484   0.000962  
  950/5000     -1581    0.2028    0.7206    0.3043     2.257      17      0.7343    0.7358    0.1484   0.000962  
  951/5000     -1592    0.2219    0.7522    0.3042     2.258     17.09    0.7329    0.73

TypeError: update() missing 1 required positional argument: 'net_weights'

## Optimize for inference

In [ ]:
from inference import create_config

ld1 = ('../../logs/2019_08_27/2019_08_27_0457/'
       'lattice8_batch100_lf10_qw00_aw10_generic_dp00')
log_dir1 = os.path.join(*ld1.split('/'))

params_file = os.path.join(log_dir1, 'parameters.pkl')
with open(params_file, 'rb') as f:
    params = pickle.load(f)
    
checkpoint_dir = os.path.join(log_dir1, 'checkpoints')
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

config, params = create_config(params)
sess = tf.Session(config=config)
saver = tf.train.import_meta_graph(f'{checkpoint_file}.meta')
saver.restore(sess, checkpoint_file)

In [ ]:
run_ops = tf.get_collection('run_ops', )
inputs = tf.get_collection('inputs')


In [ ]:
run_nodes = [node.name for node in tf.get
node_names = [node.name for node in tf.get_default_graph().as_graph_def().node]


In [ ]:
run_node_names = [op for op in run_ops if op in tf.get_default_graph().as_graph_def().node]

In [ ]:
run_node_names

In [ ]:
run_op_names = []
for op in run_ops:
    _name = op.name.split('/')
    name = ('/').join(_name[:-1])
    run_op_names.append(name)
    
run_op_names1 = [op.name for op in run_ops]
_run_op_names = (', \n').join(run_op_names)
_run_op_names1 = (', \n').join(run_op_names1)
print(_run_op_names)

In [ ]:
print(_run_op_names1)

In [ ]:
_names = 'sampler/x_update/x_out,sampler/x_update/accept_prob,observables/actions,observables/plaqs'

In [ ]:
     checkpoint_file = kwargs.get('checkpoint_file', None)
    restore_op_name = kwargs.get('restore_op_name', 'save/restore_all')
    filename_tensor = kwargs.get('filename_tensor', 'save/Const:0')

    if checkpoint_file is None:
        checkpoint_dir = os.path.dirname(input_graph)
        checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

    #  freeze_graph.freeze_graph('tensorflowModel.pbtxt', "", False,
    #                            './tensorflowModel.ckpt', "output/softmax",
    #                             "save/restore_all", "save/Const:0",
    #                             'frozentensorflowModel.pb', True, "")

    freeze_graph.freeze_graph(
        input_graph=input_graph,
        input_saver="",
        input_binary=False,  # True: `.pb` file; False: `.pbtxt` file
        input_checkpoint=checkpoint_file,
        output_node_names=output_node_names,
        restore_op_name=restore_op_name,
        filename_tensor=filename_tensor,
        output_graph=output_graph,
        clear_devices=True,
        initializer_nodes=""
    )


In [ ]:
from tensorflow.python.tools import optimize_for_inference_lib, freeze_graph

In [ ]:
from tensorflow.python.tools import optimize_for_inference_lib, freeze_graph

input_graph = os.path.join(checkpoint_dir, 'graph.pbtxt')
output_graph = os.path.join(checkpoint_dir, 'frozen_graph.pb')

restore_op_name = 'save/restore_all'
filename_tensor = 'save/Const:0'
freeze_graph.freeze_graph(
    input_graph=input_graph,
    input_saver='',
    input_binary=False,
    input_checkpoint=checkpoint_file,
    output_node_names=_names,
    restore_op_name='save/restore_all',
    filename_tensor_name='save/Const:0',
    output_graph=output_graph,
    clear_devices=True,
    initializer_nodes=''
)

In [ ]:
keys = ['x_out', 'px', 'actions_op', 'plaqs_op', 'avg_plaqs_op', 'charges_op', 'charge_diffs_op', 'eps']
_keys = (',').join(keys)
node_names = [node.name for node in tf.get_default_graph().as_graph_def().node]

In [ ]:
len(node_names)

In [ ]:
node_names[:10]

In [ ]:
%debug

In [ ]:
_node_names = [i for i in node_names if '']

In [ ]:
_node_names

In [ ]:
input_names = [op.name for op in inputs]
input_names

In [ ]:
python -m tensorflow.python.tools.freeze_graph --input_graph graph.pb --input_checkpoint test_model --output_graph graph_frozen.pb --output_node_names=y

In [ ]:
_in_names = (', ').join(input_names)
_in_names

In [ ]:
input_graph = os.path.join(checkpoint_dir, 'graph.pbtxt')
input_checkpoint = os.path.join(checkpoint_dir, 'model')
output_graph = os.path.join(checkpoint_dir, 'graph_frozen.pb')
output_node_names = (', ').join(run_op_names)

In [ ]:
%%python3 -m tensorflow.python.tools.freeze_graph \ 
    --input_graph=input_graph \
    --output_graph=output_graph \
    --output_node_names=output_node_names

In [ ]:
optimized_graph = os.path.join(checkpoint_dir, 'optimized_graph.pb')

In [ ]:
print((',').join(run_op_names))

In [ ]:
(', \n').join(input_names)

In [ ]:
%%python3 -m tensorflow.python.tools.optimize_for_inference \
    --input input_graph \
    --output_graph optimized_graph \
    --input_names (', ').join(input_names) \
    --output_names (', ').join(run_op_names)

In [ ]:
python -m tensorflow.python.tools.optimize_for_inference --input graph_frozen.pb --output graph_optimized.pb --input_names=x --output_names=y

In [ ]:
!spython3 -m tensorflow.python.tools.optimizer_for_inference --input 

## OLD

### Plot $\langle \delta_{\phi_{P}}\rangle$ vs. net_weights_arr

In [ ]:
ld1 = ('../../logs/2019_08_27/2019_08_27_0457/'
       'lattice8_batch100_lf10_qw00_aw10_generic_dp00')
log_dir1 = os.path.join(*ld1.split('/'))

In [ ]:
ld2 = ('../../logs/2019_08_27/2019_08_27_1314/'
       'lattice8_batch100_lf16_qw00_aw10_generic_dp00')
log_dir2 = os.path.join(*ld2.split('/'))

In [ ]:
ld3 = ('../../logs/cooley_logs/2019_08_28/2019_08_28_0213/'
       'lattice8_batch128_lf16_qw00_aw10_generic_dp00')
log_dir3 = os.path.join(*ld3.split('/'))

In [ ]:
ld4 = ('../../logs/2019_08_28/2019_08_28_1244/'
       'lattice8_batch100_lf16_qw00_aw10_generic_dp00')
log_dir4 = os.path.join(*ld4.split('/'))

In [ ]:
fig4, ax4 = plot_plaq_diffs_vs_net_weights(log_dir4, **kwargs)

In [ ]:
import pandas as pd
plaq_diff_txt_file = os.path.join(log_dir4, 'plaq_diffs_data.txt')
_data = pd.read_csv(plaq_diff_txt_file, header=None)
pdd = _data.values

In [ ]:
log_dir4

In [ ]:
zero_data = pdd[0]
stq_data = pdd[-1]
pdd = pdd[1:-1]

q_data = pdd[pdd[:, 2] > 0.]
t_data = pdd[pdd[:, 1] > 0.]
s_data = pdd[pdd[:, 0] > 0.]

print(q_data)

In [ ]:
print(q_data[-1])

In [ ]:
from plotters.plot_utils import plot_plaq_diffs_vs_net_weights
plt.close('all')
kwargs = {'ext': 'png'}
fig1, ax1 = plot_plaq_diffs_vs_net_weights(log_dir1, **kwargs)

In [ ]:
fig2, ax2 = plot_plaq_diffs_vs_net_weights(log_dir2, **kwargs)

In [ ]:
fig3, ax3 = plot_plaq_diffs_vs_net_weights(log_dir3, **kwargs)

In [ ]:
plaq_diff_txt_file = os.path.join(log_dir, 'plaq_diffs_data.txt')
pdd = pd.read_csv(plaq_diff_txt_file, header=None)
plaq_diff_data = pdd.values

In [ ]:
x = np.arange(25)

In [ ]:
x

In [ ]:
x[0]
x[1:9]
x[9:16]

In [ ]:
zero_weights = np.array([0.00, 0.00, 0.00])   # set weights to 0.

q_weights = np.array([[0.00, 0.00, 0.10],   # loop over Q weights
                      [0.00, 0.00, 0.25],
                      [0.00, 0.00, 0.50],
                      [0.00, 0.00, 0.75],
                      [0.00, 0.00, 1.00],
                      [0.00, 0.00, 1.50],
                      [0.00, 0.00, 2.00],
                      [0.00, 0.00, 5.00]])

t_weights = np.array([[0.00, 0.10, 0.00],
                      [0.00, 0.25, 0.00],
                      [0.00, 0.50, 0.00],
                      [0.00, 0.75, 0.00],
                      [0.00, 1.00, 0.00],
                      [0.00, 1.50, 0.00],
                      [0.00, 2.00, 0.00],
                      [0.00, 5.00, 0.00]])

s_weights = np.array([[0.10, 0.00, 0.00],
                      [0.25, 0.00, 0.00],
                      [0.50, 0.00, 0.00],
                      [0.75, 0.00, 0.00],
                      [1.00, 0.00, 0.00],
                      [1.50, 0.00, 0.00],
                      [2.00, 0.00, 0.00],
                      [5.00, 0.00, 0.00]])

stq_weights = np.array([1.00, 1.00, 1.00])

net_weights_arr = np.array([zero_weights.tolist(),
                            *q_weights.tolist(),
                            *t_weights.tolist(),
                            *s_weights.tolist(),
                            stq_weights.tolist()])
                   #*list(t_weights), *list(s_weights), list(stq_weights)]
net_weights_arr


In [ ]:
s_weights.tolist()

In [ ]:
zero_weights = [0.00, 0.00, 0.00]   # set weights to 0.

q_weights = [[0.00, 0.00, 0.10],   # loop over Q weights
             [0.00, 0.00, 0.25],
             [0.00, 0.00, 0.50],
             [0.00, 0.00, 0.75],
             [0.00, 0.00, 1.00],
             [0.00, 0.00, 1.50],
             [0.00, 0.00, 2.00],
             [0.00, 0.00, 5.00]]

t_weights = [[0.00, 0.10, 0.00],
             [0.00, 0.25, 0.00],
             [0.00, 0.50, 0.00],
             [0.00, 0.75, 0.00],
             [0.00, 1.00, 0.00],
             [0.00, 1.50, 0.00],
             [0.00, 2.00, 0.00],
             [0.00, 5.00, 0.00]]

s_weights = [[0.10, 0.00, 0.00],
             [0.25, 0.00, 0.00],
             [0.50, 0.00, 0.00],
             [0.75, 0.00, 0.00],
             [1.00, 0.00, 0.00],
             [1.50, 0.00, 0.00],
             [2.00, 0.00, 0.00],
             [5.00, 0.00, 0.00]]

stq_weights = [1.00, 1.00, 1.00]

net_weights_arr = [zero_weights, *q_weights,
                   *t_weights, *s_weights, stq_weights]

In [ ]:
net_weights_arr = np.array(net_weights_arr)
net_weights_arr

In [ ]:
nw = net_weights_arr
zero = nw[0]
q = nw[1:9]
t = nw[9:17]
s = nw[17:25]
stq = nw[25]

print(zero)
print('\n')
print(q)
print(len(q))
print('\n')
print(t)
print(len(t))
print('\n')
print(s)
print(len(s))
print('\n')
print(stq)


In [ ]:
print(plaq_diff_data[11:16])
print('\n')
print(plaq_diff_data[16])
print('\n')
print(plaq_diff_data[17])
print('\n')
print(plaq_diff_data[17:])

In [ ]:
zero_data = plaq_diff_data[0]
q_data = plaq_diff_data[1:6]
t_data = plaq_diff_data[6:11]
s_data = plaq_diff_data[11:16]
tq_data = plaq_diff_data[16]
sq_data = plaq_diff_data[17]
st_data = plaq_diff_data[18]
rand_data = plaq_diff_data[19]
stq_data = plaq_diff_data[20]

In [ ]:
qx, qy = q_data[:, 2], q_data[:, -1]
tx, ty = t_data[:, 1], t_data[:, -1]
sx, sy = s_data[:, 0], s_data[:, -1]

In [ ]:
xlabel = 'Net weight'
ylabel = 'Avg. plaq. difference'
fig, ax = plt.subplots()
ax.plot(qx, qy, marker='.', label='Transformation (Q) fn')
ax.plot(tx, ty, marker='.', label='Translation (T) fn')
ax.plot(sx, sy, marker='.', label='Scale (S) fn')
ax.plot(0, zero_data[-1], marker='s', label='S, T, Q = 0')
ax.plot(1, stq_data[-1], marker='v', label='S, T, Q = 1')
ax.set_xlabel(xlabel, fontsize=14)
ax.set_ylabel(ylabel, fontsize=14)
ax.legend(loc='best')
plt.tight_layout()
figs_dir = os.path.join(log_dir, 'figures')
out_file = os.path.join(figs_dir, 'plaq_diff_vs_net_weights.pdf')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
xlabel = 'Net weight'
ylabel = 'Avg. plaq. difference'
fig, ax = plt.subplots()
ax.plot(qx, qy, marker='.', label='Transformation (Q) fn')
ax.plot(tx, ty, marker='.', label='Translation (T) fn')
ax.plot(sx, sy, marker='.', label='Scale (S) fn')
ax.plot(0, zero_data[-1], marker='s', label='S, T, Q = 0')
ax.plot(1, stq_data[-1], marker='v', label='S, T, Q = 1')
ax.set_xlabel(xlabel, fontsize=14)
ax.set_ylabel(ylabel, fontsize=14)
ax.legend(loc='best')
plt.tight_layout()
figs_dir = os.path.join(log_dir, 'figures')
out_file = os.path.join(figs_dir, 'plaq_diff_vs_net_weights.pdf')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
plaq_diff_txt_file = os.path.join(log_dir, 'plaq_diffs_data.txt')
net_weights = []
avg_plaq_diffs = []
with open(plaq_diff_data_file, 'r') as f:
    import pdb
    pdb.set_trace()
    line = f.readline()
    vals = [float(i) for i in line.split(',')]
    net_weights.extend(vals[:2])
    avg_plaq_diff.append(vals[-1])
    

## OLDER

In [ ]:
diff_dict = {}

In [ ]:
transl_weights = [0., 0.1, 0.25, 0.5, 0.75, 1.]

In [ ]:
transl_weights[0:2]

In [ ]:
transl_weights[:3]

In [ ]:
transl_weights = [0., 0.1, 0.25, 0.5, 0.75, 1.]

offsets10_17_1036 = [0.00136, -0.00323, -0.00947, -0.01574, -0.01882, -0.02056]
diff_dict['lf10_2019_08_17_1036'] = offsets10_17_1036

offsets10_19_1946 = [0.00221, -0.00221, -0.00795, -0.01353, -0.01626, -0.01760]
diff_dict['lf10_2019_08_19_1946'] = offsets10_19_1946

offsets12_19_1713 = [-0.00047, -0.00502, -0.01164, -0.01796, -0.02116, -0.02228]
diff_dict['lf12_2019_08_19_1713'] = offsets12_19_1713

offsets15_19_2332 = [0.00043, -0.00424, -0.01042, -0.01727, -0.02072, -0.02231]
diff_dict['lf15_2019_08_19_2332'] = offsets15_19_2332

offsets16_15_1915 = [-0.00180, -0.00452, -0.00811, -0.01249, -0.01485, -0.01638]
diff_dict['lf16_2019_08_15_1915'] = offsets16_15_1915

offsets16_16_1735 = [-0.00520, -0.00819, -0.01191, -0.01593, -0.01787, -0.01887]
diff_dict['lf16_2019_08_16_1735'] = offsets16_16_1735

offsets16_17_0745 = [-0.00221, -0.00472, -0.00854, -0.01300, -0.01553, -0.01703]
diff_dict['lf16_2019_08_17_0745'] = offsets16_17_0745

offsets16_20_0052 = [0.00028, -0.00276, -0.00700, -0.01213, -0.01484, -0.01620]
diff_dict['lf16_2019_08_20_0052'] = offsets16_20_0052

offsets16_20_0504 = [0.00165, -0.00614, -0.01324, -0.01775, -0.01889, -0.01986]
diff_dict['lf16_2019_08_20_0504'] = offsets16_20_0504

offsets16_13_1546 = mean_diff_dict['2019_08_13_1546']
diff_dict['lf16_2019_08_13_1546'] = mean_diff_dict['2019_08_13_1546']

In [ ]:
diff_dict

In [ ]:
fig, ax = plt.subplots()
for key, val in diff_dict.items():
    lf_steps = int(key.split('_')[0].lstrip('lf'))
    if lf_steps == 16:
        ax.plot(transl_weights, val,
                label=r'$N_{\mathrm{LF}} = 16$')
        
ax.set_xlabel('Translation weight', fontsize=14)
ax.set_ylabel(r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$",
              #r"- \delta_{\phi_{P}}^{(\mathrm{exp})}$",
              fontsize=14)
plt.tight_layout()
ax.legend(loc='best')

In [ ]:
fig, ax = plt.subplots()
ax.plot(transl_weights, offsets10, marker='s', ls=':', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 10$')
ax.plot(transl_weights, offsets12, marker='d', ls='-.', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 12$')
ax.plot(transl_weights, offsets15, marker='v', ls='--', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 15$')
ax.plot(transl_weights, offsets16, marker='.', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_1, marker='>', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_2, marker='H', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_3, marker='^', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.set_xlabel('Translation weight', fontsize=14)
ax.set_ylabel(r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$",
              #r"- \delta_{\phi_{P}}^{(\mathrm{exp})}$",
              fontsize=14)
plt.tight_layout()
ax.legend(loc='best')

of = '../../logs/figures/avg_plaq_diff_vs_num_lf2.pdf'
out_file = os.path.join(*of.split('/'))
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
transl_weights = [0., 0.1, 0.25, 0.5, 0.75, 1.]
offsets16 = [0.00028, -0.00276, -0.00700, -0.01213, -0.01484, -0.01620]
offsets16_1 = [0.00165, -0.00614, -0.01324, -0.01775, -0.01889, -0.01986]
offsets16_2 = [-0.00180, -0.00452, -0.00811, -0.01249, -0.01485, -0.01638]
offsets16_3 = [-0.00221, -0.00472, -0.00854, -0.01300, -0.01553, -0.01703]

fig, ax = plt.subplots()
ax.plot(transl_weights, offsets16, #marker='.', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_1, #marker='>', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_2, #marker='H', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.plot(transl_weights, offsets16_3, #marker='^', ls='-', fillstyle='none',
        label=r'$N_{\mathrm{LF}} = 16$')
ax.set_xlabel('Translation weight', fontsize=14)
ax.set_ylabel(r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$",
              #r"- \delta_{\phi_{P}}^{(\mathrm{exp})}$",
              fontsize=14)
plt.tight_layout()
ax.legend(loc='best')

of = '../../logs/figures/avg_plaq_diff_vs_num_lf3.pdf'
out_file = os.path.join(*of.split('/'))
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
from scipy.stats import sem

offsets16 = np.array(
    [[0.00028, -0.00276, -0.00700, -0.01213, -0.01484, -0.01620],
     [0.00165, -0.00614, -0.01324, -0.01775, -0.01889, -0.01986],
     [-0.00180, -0.00452, -0.00811, -0.01249, -0.01485, -0.01638],
     [-0.00221, -0.00472, -0.00854, -0.01300, -0.01553, -0.01703]]
)
avg16 = offsets16.mean(axis=0)
err16 = sem(offsets16, axis=0)

In [ ]:
err16.shape

In [ ]:
from scipy.stats import sem


transl_weights = [0., 0.1, 0.25, 0.5, 0.75, 1.]
offsets10 = [0.00221, -0.00221, -0.00795, -0.01353, -0.01626, -0.01760]
offsets12 = [-0.00047, -0.00502, -0.01164, -0.01796, -0.02116, -0.02228]
offsets15 = [0.00043, -0.00424, -0.01042, -0.01727, -0.02072, -0.02231]

fig, ax = plt.subplots()
ax.plot(transl_weights, offsets10, #marker='s',
        ls=':', fillstyle='none', label=r'$N_{\mathrm{LF}} = 10$')
ax.plot(transl_weights, offsets12, #marker='d',
        ls='-.', fillstyle='none', label=r'$N_{\mathrm{LF}} = 12$')
ax.plot(transl_weights, offsets15, #marker='v',
        ls='--', fillstyle='none', label=r'$N_{\mathrm{LF}} = 15$')
ax.errorbar(transl_weights, avg16, yerr=err16, marker='', ls='-', #fillstyle='none',
            #ecolor='k', color='k',
            label=r'$N_{\mathrm{LF}} = 16$')
ax.set_xlabel('Translation weight', fontsize=14)
ax.set_ylabel(r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$",
              #r"- \delta_{\phi_{P}}^{(\mathrm{exp})}$",
              fontsize=14)
plt.tight_layout()
ax.legend(loc='best')

of = '../../logs/figures/avg_plaq_diff_vs_num_lf1.pdf'
out_file = os.path.join(*of.split('/'))
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
mean_diff_dict = {}

In [ ]:
log_dirs = [
    ('../../logs/cooley_logs/2019_08_13/2019_08_13_1546/'
     'lattice8_batch128_lf16_qw10_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_15/2019_08_15_1915/'
     'lattice8_batch128_lf16_qw10_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_16/2019_08_16_1735/'
     'lattice8_batch128_lf16_qw10_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_17/2019_08_17_0745/'
     'lattice8_batch128_lf16_qw10_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_17/2019_08_17_1036/'
     'lattice8_batch128_lf10_qw10_aw10_generic_dp05/'),
    ('../../logs/2019_08_19/2019_08_19_1713/'
     'lattice8_batch100_lf12_qw10_aw10_generic_dp05/'),
    ('../../logs/2019_08_19/2019_08_19_1946/'
     'lattice8_batch100_lf10_qw00_aw10_generic_dp05/'),
    ('../../logs/2019_08_19/2019_08_19_2332/'
     'lattice8_batch100_lf15_qw00_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_20/2019_08_20_0052/'
     'lattice8_batch128_lf16_qw00_aw10_generic_dp05/'),
    ('../../logs/cooley_logs/2019_08_20/2019_08_20_0504/'
     'lattice8_batch128_lf16_qw00_aw00_generic_dp05/'),
]

plaq_diff_dict = {}

for ld in log_dirs:
    log_dir = os.path.join(*ld.split('/'))
    params_file = os.path.join(log_dir, 'parameters.pkl')
    with open(params_file, 'rb') as f:
        params = pickle.load(f)

    runs_dirs = os.path.join(log_dir, 'runs')
    figs_dir = os.path.join(log_dir, 'figures')
    run_dirs = [os.path.join(runs_dirs, i) for i in os.listdir(runs_dirs)]
    run_strs = [i.split('/')[-1] for i in run_dirs]
    
    #run_params_files = [os.path.join(i, 'parameters.pkl') for i in run_dirs]
    #run_data_files = [os.path.join(i, 'run_data.pkl') for i in run_dirs]
    #with open(run_data_files[0], 'rb') as f:
    #    run_data = pickle.load(f)
    #with open(run_params_files[0], 'rb') as f:
    #    run_params = pickle.load(f)
        
    plotter = GaugeModelPlotter(params, figs_dir)
    
    mean_diff_arr = []
    transl_weights = []
    for d, s in zip(run_dirs, run_strs):
        run_data_file = os.path.join(d, 'run_data.pkl')
        run_params_file = os.path.join(d, 'parameters.pkl')
        with open(run_data_file, 'rb') as f:
            run_data = pickle.load(f)
        with open(run_params_file, 'rb') as f:
            run_params = pickle.load(f)

        weights = {
            'charge_weight': run_params['charge_weight'],
            'net_weights': run_params['net_weights']
        }
        xy_data, kwargs = plotter._plot_setup(run_data, 5., s, weights)
        x, y, yerr = xy_data['plaqs_diffs']
        mean_diff = np.mean(y)
        #mean_diff = plotter._plot_plaqs_diffs(xy_data['plaqs_diffs'], **kwargs)
        transl_weights.append(weights['net_weights'][1])
        #print(f"{weights['net_weights']}: {mean_diff}\n")
        #mean_diff = plotter.plot_observables(run_data, 5., s, weights)
        mean_diff_arr.append(mean_diff)

    wd_dict = dict(zip(transl_weights, mean_diff_arr))
    weights_diffs_dict = OrderedDict(sorted(wd_dict.items(),
                                            key=lambda k: k[0]))
    
    if log_dir[-1] == '/':
        log_dir = log_dir.rstrip('/')
        
    dict_str = log_dir.split('/')[-2]
    lf_steps = run_params['num_steps']
    plaq_diff_dict_key = f'lf{lf_steps}_' + dict_str
    # create key, value pair containing path to log dir
    plaq_diff_dict[plaq_diff_dict_key] = {
        'data': weights_diffs_dict,
        'log_dir': log_dir
    }
    
pd_dict = OrderedDict(sorted(plaq_diff_dict.items(),
                             key=lambda k: int(k[0].split('_')[0].lstrip('lf'))))

In [ ]:
lf16_yarr

In [ ]:
#lf16_xarr = []
lf10_yarr = []
lf12_yarr = []
lf15_yarr = []
lf16_yarr = []
fig, ax = plt.subplots()
for key, val in pd_dict.items():
    lf_steps = int(key.split('_')[0].lstrip('lf'))
    xy_data = val['data']
    x = list(xy_data.keys())
    y = list(xy_data.values())
    if lf_steps == 10:
        lf10_yarr.append(y)
    if lf_steps == 12:
        lf12_yarr.append(y)
    if lf_steps == 15:
        lf15_yarr.append(y)
    if lf_steps == 16:
        lf16_yarr.append(y)
        
# N_lf = 10
lf10_yavg = np.mean(np.array(lf10_yarr), axis=0)
lf10_yerr = sem(np.array(lf10_yarr), axis=0)
ax.errorbar(x, lf10_yavg, yerr=lf10_yerr,  capthick=2., capsize=2., 
            ls=':', label=r'$N_{\mathrm{LF}} = $' + '10')

# N_lf = 12
ax.plot(x, lf12_yarr[0], ls='-.', label=r'$N_{\mathrm{LF}} = $' + '12')

# N_lf = 15
ax.plot(x, lf15_yarr[0], ls='--', label=r'$N_{\mathrm{LF}} = $' + '15')
    
# N_lf = 16
lf16_yavg = np.mean(np.array(lf16_yarr), axis=0)
lf16_yerr = sem(np.array(lf16_yarr), axis=0)
ax.errorbar(x, lf16_yavg, yerr=lf16_yerr, capthick=1.5, capsize=1.5,
            label=r'$N_{\mathrm{LF}} = $' + '16')
        
xlabel = 'Translation weight'
ylabel = r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$"
ax.grid(True)
ax.set_xlabel(xlabel, fontsize=14)
ax.set_ylabel(ylabel, fontsize=14)
plt.tight_layout()
ax.legend(loc='best')

of = '../../logs/figures/avg_plaq_diff_vs_transl_weight_errs.pdf'
#of = '../../logs/figures/avg_plaq_diff_vs_transl_weight_lf10_12_15.pdf'
#of = '../../logs/figures/avg_plaq_diff_vs_transl_weight_lf16.pdf'
out_file = os.path.join(*of.split('/'))
plt.savefig(out_file, dpi=400, bbox_inches='tight')
        
#ax.set_xlabel('Translation weight', fontsize=14)
#ax.set_ylabel(r"$\langle\delta_{\phi_{P}}^{(\mathrm{obs})}\rangle$",
#              fontsize=14)
#plt.tight_layout()
#ax.legend(loc='best')

In [ ]:
debug

In [ ]:
ld = ('../../logs/cooley_logs/2019_08_13/2019_08_13_1546/'
      'lattice8_batch128_lf16_qw10_aw10_generic_dp05')
log_dir = os.path.join(*ld.split('/'))

params_file = os.path.join(log_dir, 'parameters.pkl')
with open(params_file, 'rb') as f:
    params = pickle.load(f)
    
runs_dirs = os.path.join(log_dir, 'runs')
figs_dir = os.path.join(log_dir, 'figures')
run_dirs = [os.path.join(runs_dirs, i) for i in os.listdir(runs_dirs)]
run_strs = [i.split('/')[-1] for i in run_dirs]
run_strs
run_dirs

In [ ]:
run_params_files = [os.path.join(i, 'parameters.pkl') for i in run_dirs]
run_data_files = [os.path.join(i, 'run_data.pkl') for i in run_dirs]

In [ ]:
with open(run_data_files[0], 'rb') as f:
    run_data = pickle.load(f)
with open(run_params_files[0], 'rb') as f:
    run_params = pickle.load(f)
run_params['net_weights']

In [ ]:
plotter = GaugeModelPlotter(run_params, figs_dir)

In [ ]:
dict_str = log_dir.split('/')[-2]
lf_steps = run_params['num_steps']
dict_key = f'lf{lf_steps}_' + dict_str

In [ ]:
mean_diff_arr = []
transl_weights = []
for d, s in zip(run_dirs, run_strs):
    run_data_file = os.path.join(d, 'run_data.pkl')
    run_params_file = os.path.join(d, 'parameters.pkl')
    with open(run_data_file, 'rb') as f:
        run_data = pickle.load(f)
    with open(run_params_file, 'rb') as f:
        run_params = pickle.load(f)
        
    weights = {
        'charge_weight': run_params['charge_weight'],
        'net_weights': run_params['net_weights']
    }
    xy_data, kwargs = plotter._plot_setup(run_data, 5., s, weights)
    mean_diff = plotter._plot_plaqs_diffs(xy_data['plaqs_diffs'], **kwargs)
    transl_weights.append(weights['net_weights'][1])
    #print(f"{weights['net_weights']}: {mean_diff}\n")
    #mean_diff = plotter.plot_observables(run_data, 5., s, weights)
    mean_diff_arr.append(mean_diff)
    
wd_dict = dict(zip(transl_weights, mean_diff_arr))
weights_diffs_dict = OrderedDict(sorted(weights_diffs_dict.items(),
                                        key=lambda k: k[0]))
    
dict_str = log_dir.split('/')[-2]
lf_steps = run_params['num_steps']
dict_key = f'lf{lf_steps}_' + dict_str
diff_dict[dict_key] = mean_diff_arr
#mean_diff_dict[dict_str] = mean_diff_arr

In [ ]:
from collections import OrderedDict
#OrderedDict(sorted(d.items(), key=lambda t: t[1]))
weights_diffs_dict =  dict(zip(transl_weights, mean_diff_arr))
weights_diffs_dict = OrderedDict(sorted(weights_diffs_dict.items(),
                                        key=lambda k: k[0]))
weights_diffs_dict

In [ ]:
transl_weights
mean_diff_arr

In [ ]:
plotter.plot_observables(run_data, 5., run_strs[0], weights)

In [ ]:
xy_data = plotter._parse_data(run_data, 5.)

In [ ]:
kwargs = {
    'markers': False,
    'lines': True,
    'alpha': 0.6,
    'legend': False,
    'ret': False,
    'out_file': []
}

In [ ]:
plotter._plot_plaqs_diffs(xy_data['plaqs_diffs'], **kwargs)

In [ ]:
'../args/params.pkl'
params_file = os.path.join('..', 'args', 'params.pkl')
with open(params_file, 'rb') as f:
    params = pickle.load(f)

In [ ]:
from inference import create_config

checkpoint_dir = os.path.join(params['log_dir'], 'checkpoints/')
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

config, params = create_config(params)
sess = tf.Session(config=config)
saver = tf.train.import_meta_graph(f'{checkpoint_file}.meta')
saver.restore(sess, checkpoint_file)

In [ ]:
autocorrs = np.array(run_data['charges_autocorrs'])
autocorrs_avg = np.mean(autocorrs, axis=0)
num_steps = autocorrs.shape[1]
mid = num_steps // 2
lower = int(mid - num_steps * 0.1)
upper = int(mid + num_steps * 0.1)

fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, sharex=False)
ax0.plot(np.arange(len(autocorrs_avg)), autocorrs_avg)
ax1.plot(np.arange(lower, upper), autocorrs_avg[lower:upper])

In [ ]:
from plotters.gauge_model_plotter import GaugeModelPlotter
import utils.file_io as io

figs_dir = os.path.join(log_dir, 'figures')
io.check_else_make_dir(figs_dir)
run_str = 'steps_10000_beta_50_eps_016_qw_10_0'

weights = {
    'charge_weight': params['charge_weight'],
    'net_weights': [1., 1., 1.]
}

plotter = GaugeModelPlotter(params, figs_dir)
#plotter.plot_observables(run_data, beta=6., run_str=run_str, weights=weights)

In [ ]:
xy_data = plotter._parse_data(run_data, beta=6.)

In [ ]:
plotter._plot_plaqs(xy_data['plaqs'], beta=6., save=False)

In [ ]:
plotter.plot_observables(run_data, beta=6., run_str=run_str, weights=weights)

In [ ]:
plt.errorbar()

In [ ]:
from utils.attr_dict import AttrDict
#ld = ('../../logs/2019_07_23/2019_07_23_1349/'
#      'lattice8_batch128_lf6_qw10_aw10_conv2D_dp00_bn/')
#log_dir = os.path.join(*ld.split('/'))
#params_file = os.path.join(log_dir, 'parameters.pkl')
pf = '../args/params.pkl'
params_file = os.path.join(*pf.split('/'))
#params_file = os.path.join('..', '', 'params.pkl')
with open(params_file, 'rb') as f:
    params = pickle.load(f)
params    
params['data_format'] = 'channels_last'

#for key, val in params.items():
#    FLAGS.__dict__[key] = val
    
#FLAGS = AttrDict(params.items())

In [ ]:
params['log_dir'] = None

import utils.file_io as io
log_dir = io.create_log_dir(params)

params['log_dir'] = io.create_log_dir(params)

In [ ]:
checkpoint_dir = os.path.join(params['log_dir'], 'checkpoints/')
io.check_else_make_dir(checkpoint_dir)

model = GaugeModel(params=params)

In [ ]:
from variables import TF_FLOAT, NP_FLOAT, GLOBAL_SEED
from loggers.train_logger import TrainLogger
from inference import create_config

train_logger = TrainLogger(model, log_dir, params['summaries'])
config, params = create_config(params)

charge_weight_init = params['charge_weight']
net_weights_init = [1., 1., 1.]
samples_init = np.reshape(np.array(model.lattice.samples, dtype=NP_FLOAT),
                          (model.num_samples, model.x_dim))
beta_init = model.beta_init

init_feed_dict = {
    model.x: samples_init,
    model.beta: beta_init,
    model.charge_weight: charge_weight_init,
    model.net_weights[0]: net_weights_init[0],  # scale_weight
    model.net_weights[1]: net_weights_init[1],  # transformation_weight
    model.net_weights[2]: net_weights_init[2],  # translation_weight
    model.train_phase: True,
}

target_collection = []
collection = tf.local_variables() + target_collection

local_init_op = tf.variables_initializer(collection)
ready_for_local_init_op = tf.report_uninitialized_variables(collection)
init_op = tf.global_variables_initializer()

scaffold = tf.train.Scaffold(
    init_feed_dict=init_feed_dict,
    local_init_op=local_init_op,
    ready_for_local_init_op=ready_for_local_init_op
)

In [ ]:
# ----------------------------------------------------------
#                       TRAINING
# ----------------------------------------------------------
hooks = []
sess_kwargs = {
    'checkpoint_dir': checkpoint_dir,
    'scaffold': scaffold,
    'hooks': hooks,
    'config': config,
    'save_summaries_secs': None,
    'save_summaries_steps': None
}

sess = tf.train.MonitoredTrainingSession(**sess_kwargs)
tf.keras.backend.set_session(sess)
sess.run(init_op)

trainer = GaugeModelTrainer(sess, model, train_logger)
train_kwargs = {
    'samples_np': samples_init,
    'beta_np': beta_init,
    'net_weights': net_weights_init
}

trainer.train(20, **train_kwargs)

sess.close()
tf.reset_default_graph()

In [ ]:
from inference import create_config
from loggers.run_logger import RunLogger
from plotters.gauge_model_plotter import GaugeModelPlotter

checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
config, params = create_config(params)

sess = tf.Session(config=config)

saver = tf.train.import_meta_graph(f'{checkpoint_file}.meta')
saver.restore(sess, checkpoint_file)

run_ops = tf.get_collection('run_ops')
inputs = tf.get_collection('inputs')

run_logger  = RunLogger(params, inputs, run_ops, save_lf_data=False)
plotter = GaugeModelPlotter(params, run_logger.figs_dir)

In [ ]:
from inference import inference_setup
from runners.runner import GaugeModelRunner
from inference import run_inference

kwargs = {
    'run_steps': 100,
    'loop_net_weights': True,
    'plot_lf': True
}
params.update(kwargs.items())

inference_dict = inference_setup(params)

runner = GaugeModelRunner(sess, params, inputs, run_ops, run_logger)
run_inference(inference_dict, runner, run_logger, plotter)

In [ ]:
runner.eps

In [ ]:
tf.global_variables()

In [ ]:
tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
checkpoint_dir = os.path.join(params['log_dir'], 'checkpoints')
sess = tf.Session(config=config)
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir))
run_logger = RunLogger(model, params['log_dir'], save_lf_data=False)
plotter = GaugeModelPlotter(model, run_logger.figs_dir)

In [ ]:
xnet_x, xnet_v, generic_net = model.dynamics.x_fn.layers

In [ ]:
xconv_x1 = xnet_x.layers[0]

In [ ]:
k, b = xconv_x1.get_weights()
k_rand = np.random.randn(*k.shape)
b_rand = np.random.randn(*b.shape)

xconv_x1.set_weights([k_rand, b_rand])

In [ ]:
xconv_x1.set_weights

In [ ]:
xnet = model.dynamics.x_fn
vnet = model.dynamics.v_fn

for xblock, vblock in zip(xnet.layers, vnet.layers):
    for xlayer, vlayer in zip(xblock.layers, vblock.layers):
        try:
            print(f'xlayer.name: {xlayer.name}')
            print(f'vlayer.name: {vlayer.name}')
            kx, bx = xlayer.get_weights()
            kv, bv = vlayer.get_weights()
            kx_rand = np.random.randn(*kx.shape)
            bx_rand = np.random.randn(*bx.shape)
            kv_rand = np.random.randn(*kv.shape)
            bv_rand = np.random.randn(*bv.shape)

            xlayer.set_weights([kx, bx])
            vlayer.set_weights([kv, bv])
        except ValueError:
            print(f'Unable to set weights for: {xlayer.name}')
            print(f'Unable to set weights for: {vlayer.name}')
        

In [ ]:
xconv_x1w, xconv_x1b = xconv_x1.get_weights()

In [ ]:
random_sample = np.random.randn(*model.lattice.samples.shape)

In [ ]:
random_sample = np.array(np.random.randn(*model.lattice.samples.shape), dtype=np.float32)
samples_reshaped = tf.reshape(tf.convert_to_tensor(random_sample), (-1, 8, 8, 2))
xconv_out1 = model.dynamics.x_fn.x_conv_net.conv1(samples_reshaped)

In [ ]:
xconv_out1.shape

In [ ]:
for f in range(xconv_out1.shape[-1]):
    fig, ax = plt.subplots()
    ax.imshow(sess.run(xconv_out1[0, :, :, f]))

In [ ]:
xconv1w, xconv1b = model.dynamics.x_fn.x_conv_net.conv1.get_weights()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
for c in range(xconv1w.shape[2]):
    for f in range(xconv1w.shape[-1]):
        fig, ax = plt.subplots()
        ax.imshow(xconv1w[:, :, c, f])

In [ ]:
model.dynamics.x_fn.x_conv_net.conv1.get_output_at(0)

In [ ]:
sess.run(model.dynamics.x_fn.x_conv_net.conv1, feed_dict=init_feed_dict)

In [ ]:
train_logger = TrainLogger(model, model.log_dir, FLAGS.summaries)

In [ ]:
from globals import TF_FLOAT, NP_FLOAT, GLOBAL_SEED

charge_weight_init = FLAGS.charge_weight
net_weights_init = [1., 1., 1.]
samples_init = np.reshape(np.array(model.lattice.samples, dtype=NP_FLOAT),
                          (model.num_samples, model.x_dim))
beta_init = model.beta_init

init_feed_dict = {
    model.x: samples_init,
    model.beta: beta_init,
    model.charge_weight: charge_weight_init,
    model.net_weights[0]: net_weights_init[0],  # scale_weight
    model.net_weights[1]: net_weights_init[1],  # transformation_weight
    model.net_weights[2]: net_weights_init[2],  # translation_weight
    model.train_phase: True,
}

In [ ]:
x_layers_dict = {
    (idx, l.name): l for idx, l in enumerate(model.dynamics.x_fn.layers)
}

xx_conv_layers = {
    l.name: l for l in model.dynamics.x_fn.x_conv_block.layers
}
xv_conv_layers = {
    l.name: l for l in model.dynamics.x_fn.v_conv_block.layers
}
x_generic_layers = {
    l.name: l for l in model.dynamics.x_fn.generic_block.layers
}
xx_conv_layers
xv_conv_layers
x_generic_layers

In [ ]:
q = tf.reshape(model.x, (-1, *model.lattice.samples.shape[1:], 1))
xx_conv_layers['conv1'].compute_output_shape(q.shape)

In [ ]:
q = tf.placeholder(dtype=TF_FLOAT, shape=model.x.shape, name='q')
if model.x.shape != model.dynamics.x_fn._input_shape[1:]:
    q = tf.reshape(q, (-1, *model.dynamics.x_fn._input_shape[1:]))
q.shape

In [ ]:
xnet_xlayers = []
xnet_vlayers = []
for layer in model.dynamics.x_fn.layers:
    if 'conv_x' in layer.name:
        xnet_xlayers.append(layer)
    if 'conv_v' in layer.name:
        xnet_vlayers.append(layer)
        
for 

In [ ]:
x_net_x_layers = model.dynamics.x_fn.layers

In [ ]:
target_collection = []
collection = tf.local_variables() + target_collection

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./model.h5', verbose=1)
]

In [ ]:
local_init_op = tf.variables_initializer(collection)
ready_for_local_init_op = tf.report_uninitialized_variables(collection)

In [ ]:
import utils.file_io as io

checkpoint_dir = os.path.join(model.log_dir, 'checkpoints')
io.check_else_make_dir(checkpoint_dir)

scaffold = tf.train.Scaffold(
    init_feed_dict=init_feed_dict,
    local_init_op=local_init_op,
    ready_for_local_init_op=ready_for_local_init_op
)

sess_kwargs = {
    'checkpoint_dir': checkpoint_dir,
    'scaffold': scaffold,
    'hooks': [],
    'config': config,
    'save_summaries_secs': None,
    'save_summaries_steps': None
}

sess = tf.train.MonitoredTrainingSession(**sess_kwargs)

In [ ]:
trainer = GaugeModelTrainer(sess, model, train_logger)

In [ ]:
tf.keras.backend.set_learning_phase(True)

In [ ]:
tf.keras.backend.learning_phase()

In [ ]:
v_rand = tf.random_normal(tf.shape(model.x), seed=GLOBAL_SEED)
t = model.dynamics._get_time(0, tile=tf.shape(model.x)[0])

mask, mask_inv = model.dynamics._get_mask(0)

x_scale, x_translation, x_transformation = model.dynamics.x_fn(
    (v_rand, mask * model.x, t), model.train_phase
)

dynamics_out = model.dynamics(model.x, model.beta, model.net_weights, model.train_phase)

In [ ]:
feed_dict = {
    model.x: samples_init,
    model.beta: beta_init,
    model.charge_weight: charge_weight_init,
    model.net_weights[0]: net_weights_init[0],  # scale_weight
    model.net_weights[1]: net_weights_init[1],  # transformation_weight
    model.net_weights[2]: net_weights_init[2],  # translation_weight
    model.train_phase: True,
}

In [ ]:
dynamics_out = model.dynamics(model.x, model.beta, 
                              model.net_weights, model.train_phase)

In [ ]:
from network.network_utils import batch_norm

x_reshaped = model.dynamics.x_fn.reshape_5D(model.x)
v_reshaped = model.dynamics.v_fn.reshape_5D(v_rand)

conv_x1 = model.dynamics.x_fn.conv_x1(x_reshaped)
max_pool_x1 = model.dynamics.x_fn.max_pool_x1(conv_x1)
conv_x2 = model.dynamics.x_fn.conv_x2(max_pool_x1)
bn_x = batch_norm(conv_x2, model.train_phase,
                  axis=model.dynamics.x_fn.bn_axis,
                  internal_update=True)
max_pool_x2 = model.dynamics.x_fn.max_pool_x2(tf.nn.relu(bn_x))

conv_v1 = model.dynamics.x_fn.conv_v1(v_reshaped)
max_pool_v1 = model.dynamics.x_fn.max_pool_v1(conv_v1)
conv_v2 = model.dynamics.x_fn.conv_v2(max_pool_v1)
bn_v = batch_norm(conv_v2, model.train_phase,
                  axis=model.dynamics.x_fn.bn_axis,
                  internal_update=True)
max_pool_v2 = model.dynamics.x_fn.max_pool_v2(tf.nn.relu(bn_v))

x_flat = model.dynamics.x_fn.flatten(max_pool_x2)
v_flat = model.dynamics.x_fn.flatten(max_pool_v2)

x_out = tf.nn.relu(model.dynamics.x_fn.x_layer(x_flat))
v_out = tf.nn.relu(model.dynamics.x_fn.v_layer(v_flat))
t_out = tf.nn.relu(model.dynamics.x_fn.t_layer(t))

h1 = tf.nn.relu(x_out + v_out + t_out)
h2 = tf.nn.relu(model.dynamics.x_fn.h_layer(h))

translation = model.dynamics.x_fn.translation_layer(h)
scale = (tf.nn.tanh(model.dynamics.x_fn.scale_layer(h))
         * tf.exp(model.dynamics.x_fn.coeff_scale))

transformation = (model.dynamics.x_fn.transformation_layer(h)
                  * tf.exp(model.dynamics.x_fn.coeff_transformation))

layers = {
    'conv_x12': conv_x12,
    'conv_x1': conv_x1,
    'max_pool_x1': max_pool_x1,
    'conv_x2': conv_x2,
    'bn_x': bn_x,
    'max_pool_x2': max_pool_x2,
    'x_flat': x_flat,
    'x_out': x_out,
    't_out': t_out,
    'h1': h1,
    'h2': h2,
}

print(f'model.lattice.samples.shape: {model.lattice.samples.shape}\n')
print(f'model.x.shape: {model.x.shape}\n')
print(f'x_reshaped.shape: {x_reshaped.shape}\n')
for name, layer in layers.items():
      print(f'{name}: {layer.shape}\n')